In [221]:
import pandas as pd
df_train = pd.read_csv('train.csv', encoding='utf-8')
df_test = pd.read_csv('test.csv', encoding='utf-8')

In [222]:
import re
def preprocessor(text):
    text = re.sub('[\W]+', ' ', text.lower())
    return text

In [223]:
df_train['text'] = df_train['text'].apply(preprocessor)

In [224]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df_train['author'] = class_le.fit_transform(df_train['author'].values)

In [225]:
from sklearn.model_selection import train_test_split
X, y = df_train.iloc[:, df_train.columns.get_loc('text')].values, \
       df_train.iloc[:, df_train.columns.get_loc('author')].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   stratify=y)

In [226]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords as sw
stopwords = sw.words('english')

[nltk_data] Downloading package stopwords to /home/gg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [227]:
def tokenizer(text):
    return text.split()

In [228]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [229]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def tokenizer_lemma(text):
    return [lemma.lemmatize(word) for word in text.split()]

In [230]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

param_grid = {'vect__ngram_range': [(2,2)],
              'vect__stop_words': [stopwords, None],
              'vect__tokenizer': [tokenizer,
                                  tokenizer_porter,
                                  tokenizer_lemma],
              'vect__norm': ['l1', 'l2', None],
              'vect__use_idf':[False, True],
              'clf__penalty': ['l1', 'l2'],
              'clf__C': [0.01]}

pipe = Pipeline([
                   ('vect', tfidf),
                   ('clf', LogisticRegression(random_state=42))
                   ])

model = RandomizedSearchCV(pipe, param_grid, scoring='neg_log_loss', cv=10, n_iter=25, verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.8min finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params=None, iid=True, n_iter=25, n_jobs=-1,
          param_distributions={'vect__ngram_range': [(2, 2)], 'vect__norm': ['l1', 'l2', None], 'vect__stop_words': [[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u"you're", u"you've", u"you'll", u"you'd", u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'...at 0x7ff13509e8c0>], 'vect__use_idf': [False, True], 'clf__C': [0.01], 'clf__penalty': ['l1', 'l2']},
          pre_dispatch='2*

In [231]:
print(model.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), norm=None, preprocessor=None, smooth_idf=True...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [232]:
print(model.best_params_)

{'vect__ngram_range': (2, 2), 'vect__norm': None, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7ff13509ea28>, 'vect__use_idf': False, 'clf__C': 0.01, 'clf__penalty': 'l2'}


In [233]:
print(model.best_score_)

-0.955032009791158


In [234]:
y_pred_train = model.best_estimator_.predict(X_train)

In [235]:
from sklearn.metrics import confusion_matrix
conf_matrix_train = confusion_matrix(y_true=y_train, y_pred=y_pred_train)
print(conf_matrix_train)

[[5240  106  184]
 [1744 1974  226]
 [1536  123 2572]]


In [236]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred_train)

0.7140459686245896

In [237]:
y_pred_test = model.best_estimator_.predict(X_test)

In [238]:
conf_matrix_test = confusion_matrix(y_true=y_test, y_pred=y_pred_test)
print(conf_matrix_test)

[[2135   81  154]
 [ 981  565  145]
 [ 914  116  783]]


In [239]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_test)

0.5929519918283963

In [240]:
import pickle
import os
pickle.dump(model.best_estimator_,
           open(os.path.join('./pkl_objects', 'tfidf_2gram_lr_pipe.pkl'), 'wb'),
           protocol=2)